In [ ]:
import util
from forward import solve_forward
from simulation import Simulation
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
# to load large .mat files
import mat73
import random
from net import EEG_CNN

### Train the CNN

In [ ]:
eeg_topos = mat73.loadmat('/media/thanos/Elements/thanos/sim_data/sim_type_1/eeg_topographies.mat')['eeg_topos']
eeg_topos = eeg_topos.transpose(2, 0, 1)

In [ ]:
topo_idx = random.randint(0,eeg_topos.shape[0]-1)
topo = eeg_topos[topo_idx,:,:]
plt.contourf(topo, cmap=cm.get_cmap('viridis'))
cbar = plt.colorbar()
plt.draw()
plt.title('Topography for eeg signal: {}'.format(topo_idx+1))

In [ ]:
# load previous simulation
# sim = util.load_object('/media/thanos/Elements/thanos/sim_data/sim_type_1/sim_obj_100k.pkl')
fwd = solve_forward()
sim = Simulation(fwd=fwd)
sim.simulate(n_samples=100)

In [ ]:
eegcnn = EEG_CNN(sim=sim, eeg_topographies=eeg_topos)

In [ ]:
eegcnn.build_model()

In [ ]:
eegcnn.fit(patience=35)

In [ ]:
#eegcnn.save_nn('/media/thanos/Elements/thanos/nn_trained/cnn_model_4.h5')

eegcnn.save_nn('cnn_model_4.h5')

### Evaluate the CNN

In [ ]:
eegcnn.load_nn('/media/thanos/Elements/thanos/nn_trained/cnn_model_4.h5')

In [ ]:
eeg_topos_eval = mat73.loadmat('/media/thanos/Elements/thanos/sim_data/sim_type_2/eeg_topographies_type_2.mat')['eeg_topos']
eeg_topos_eval = eeg_topos_eval.transpose(2, 0, 1)

sim_type_2 = util.load_object('/media/thanos/Elements/thanos/sim_data/sim_type_2/sim_obj_10k_type_2.pkl')
sources_eval = sim_type_2.source_data

In [ ]:
mse = eegcnn.evaluate_mse(eeg=eeg_topos_eval,sources=sources_eval)
nmse = eegcnn.evaluate_nmse(eeg=eeg_topos_eval, sources=sources_eval)

print('Normilized MSE:', np.mean(nmse))
print('MSE:', np.mean(mse))